In [30]:
import tushare as ts
import pymongo
import datetime

In [31]:
client = pymongo.MongoClient("127.0.0.1", 27017)


In [104]:
def getHistoryData(code, startDate, endDate):
    res = ts.get_k_data(code, start=startDate, end=endDate)
    db = client.Trade
    collection = db[code]
    for k, v in res.iterrows():
        date = datetime.datetime.strptime(str(v.date), "%Y-%m-%d")
        item = {
            "date": date,
            "open": v.open,
            "close": v.close,
            "high": v.high,
            "low": v.low,
            "volume": v.volume
        }

        collection.replace_one({"date": date}, item, upsert=True)
    
    print("%s - %s rows" % (code, res.shape[0]))

In [ ]:
collection = client.Common.Stock
today = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
stocks = collection.find({})
for v in stocks:
    code = v["code"]
    if "kEndDate" in v:
        date = v["kEndDate"] + datetime.timedelta(days=-1)
    elif "foundTime" in v:
        date = v["foundTime"]
    else:
        date = datetime.datetime(2000, 1, 1)
    getHistoryData(code, date.strftime("%Y-%m-%d"))
    collection.update_one({"code": code}, {"$set": {"kEndDate": today}})